In [1]:

import scanpy as sc, anndata as ad, numpy as np, pandas as pd
from scipy import sparse
from anndata import AnnData
import warnings
import socket
from matplotlib import pylab
import sys
import yaml
import os
import matplotlib.pyplot as plt
import scvelo as scv
from pybiomart import Server
from tqdm.notebook import tqdm
import itertools
import random

warnings.filterwarnings('ignore')

In [2]:
get_ipython().run_line_magic('matplotlib', 'inline')

In [3]:

sc.settings.verbosity = 3         # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=50, facecolor='white')
pylab.rcParams['figure.figsize'] = (4, 4)

scanpy==1.8.1 anndata==0.7.6 umap==0.4.6 numpy==1.20.2 scipy==1.6.3 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.13.1 python-igraph==0.9.8 louvain==0.7.1 pynndescent==0.5.5


In [4]:
outdir="./outdir"

In [5]:
externalDataNowa = outdir+"/BadhuriCurated.h5ad"
externalDataNowaraw = "./data/Badhuri_atlas.h5ad"

In [6]:

# Extract metadata and combine with raws
adata = sc.read_h5ad(externalDataNowaraw)
#re index with symbols

ENSG_to_SYMBOL = adata.var.copy()


adata.var.index = adata.var.feature_name
adata.var.index.name = None
obsMD = sc.read_h5ad(externalDataNowa).raw.to_adata().obs


#Subset raw obs with processed obs
adata = adata[obsMD.index]


# OBS metadata
colsToImport = [col for col in obsMD.columns.tolist() if col not in adata.obs.columns.tolist()]
obsMD = obsMD[colsToImport]
adata.obs = pd.concat([adata.obs, obsMD[colsToImport]], axis = 1)


adataRaw = adata.raw.to_adata().copy()
adataRaw.var = pd.concat([adataRaw.var,ENSG_to_SYMBOL], axis = 1)
adataRaw.var.index = adataRaw.var.feature_name
adataRaw.var.index.name = None
adataRaw.var = adataRaw.var[["feature_name"]]
adataRaw.var = adataRaw.var.drop(columns="feature_name")


# Replace processed X with raws .raw of *prebulk files will contain norm and logged counts
adata.X = adataRaw[adata.obs_names,adata.var_names ].X.copy()
adata.raw = sc.pp.log1p(sc.pp.normalize_total(adataRaw,copy=True), copy = True).copy()
adata.var = adata.var[["feature_name"]]
adata.var = adata.var.drop(columns="feature_name")

normalizing counts per cell
    finished (0:00:02)


In [7]:
adata.layers["mint"] = adata.X.copy()

sc.pp.normalize_total(adata)

normalizing counts per cell
    finished (0:00:02)


In [8]:
# In[ ]:
#Geschw classes : 
# ExN      117523
# ExM      102893
# ExDp1     18358
# ExM-U     12528
# ExDp2       707

adataBU = adata.copy()




#1) Focus on maturingExcitatory neurons  regardless gw WE ENTIRELY RMEOVED BATCH1

adata = adataBU.copy()
FinaLeaf="/Neurons"
if not os.path.isdir(outdir+FinaLeaf):
    os.mkdir(outdir+FinaLeaf)
##ExM ANALYSIS
##ExM ANALYSIS
##ExM ANALYSIS
adata.obs["development_stage_new"] = adata.obs["development_stage"].str.split(" ", expand=True)[0]   
adata = adata[(adata.obs["GeschIngestedAnno"] == "maturingExcitatory") & (adata.obs["cluster_label_aggr"] == "excitatory") & (adata.obs["batch"] != "Batch1") ]
#check previous version -> sc.read_h5ad("./outdir/Neurons/Badhuri_neurons_prebulk.Curated.h5ad")
adata.write_h5ad(outdir+FinaLeaf+"/Badhuri_Neurons_prebulk.Curated.h5ad")



#8) Cycling only
adata = adataBU.copy()
FinaLeaf="/Cycling"
##dividing ANALYSIS
##dividing ANALYSIS
##dividing ANALYSIS
adata.obs["development_stage_new"] = adata.obs["development_stage"].str.split(" ", expand=True)[0]   

barcodelist = pd.read_csv(outdir+FinaLeaf+"/Badhuri_cycling_prebulk.Curated.barcodelist.tsv", sep="\t")["0"].tolist()
adata = adata[adata.obs_names.isin(barcodelist)]
#check previous version -> sc.read_h5ad("./outdir/Cycling/Badhuri_cycling_prebulk.Curated.h5ad")
adata.write_h5ad(outdir+FinaLeaf+"/Badhuri_cycling_prebulk.Curated.h5ad")




#9) Progenitors only
adata = adataBU.copy()
FinaLeaf="/Progenitors"
##Progenitors ANALYSIS
##Progenitors ANALYSIS
##Progenitors ANALYSIS
adata.obs["development_stage_new"] = adata.obs["development_stage"].str.split(" ", expand=True)[0]   

barcodelist = pd.read_csv(outdir+FinaLeaf+"/Badhuri_progenitors_prebulk.Curated.barcodelist.tsv", sep="\t")["0"].tolist()
adata = adata[adata.obs_names.isin(barcodelist)]
#check previous version -> sc.read_h5ad("./outdir/Progenitors/Badhuri_progenitors_prebulk.Curated.h5ad")
adata.write_h5ad(outdir+FinaLeaf+"/Badhuri_progenitors_prebulk.Curated.h5ad")






Trying to set attribute `.obs` of view, copying.
... storing 'development_stage_new' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'development_stage_new' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'development_stage_new' as categorical
